<a href="https://colab.research.google.com/github/UniVR-DH/DBMS-course/blob/main/notebooks/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test of SQLite

> SQLite is a Recommended Storage Format for datasets according to the US Library of Congress.  See: https://www.sqlite.org/draft/locrsf.html


SQLite is a compact, self-contained database system that allows users to easily organize and analyze data without needing to set up or manage separate database servers. Its simplicity and broad compatibility make it perfect for managing various types of information, from text collections to project metadata.

SQLite's extreme portability means it comes pre-installed on most operating systems and devices, making it readily available for use in a wide range of applications and projects.

## Connecting to SQLite Database

The following Python code establishes a connection to a SQLite database named "lecture01.db" and creates a cursor object for executing SQL commands:

In [1]:
import sqlite3
conn = sqlite3.connect("lecture01.db")
cursor = conn.cursor()

**Warning for Google Colab users:** If you're running this code in Colab, be aware that the database file will be created in the temporary runtime environment and will not persist after the session ends.

For persistent storage in Colab, consider using Google Drive integration or download the file and upload it when needed.

## Creating a Table

The next command creates a table named "book" with columns for book details, and the Python code executes the command and commits the changes:

In [2]:
cursor.execute('''
CREATE TABLE book (
    book_id INTEGER PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    publisher_name VARCHAR(100),
    publication_year INTEGER,
    language CHAR(2)
);
''')

conn.commit()
print("Ready!")

Ready!


## Listing All Tables in SQLite Database

This Python code retrieves and displays all table names in the connected SQLite database:

This snippet demonstrates how to query SQLite's system table `sqlite_master` to get a list of all tables in the database, then prints each table name.

It's useful for database exploration and verification of table creation.

In [3]:
# Query to get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Print the table names
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
book


## Creating Author and Book-Author Relationship Tables

This SQL code creates two new tables: "author" for storing author information, and "book_author" for managing the many-to-many relationship between books and authors:

The second command declares a FOREIGN KEY, demonstrating a common database design pattern for representing many-to-many relationships.

In [4]:
cursor.execute('''
CREATE TABLE author (
    author_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    nationality VARCHAR(50)
)
''')

cursor.execute('''
CREATE TABLE book_author (
    book_id INTEGER,
    author_id INTEGER,
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES book(book_id),
    FOREIGN KEY (author_id) REFERENCES author(author_id)
)
''')

conn.commit()
print("Ready!")

Ready!


# INSERTING DATA

In [5]:
# Insert books
cursor.execute('''
INSERT INTO book (book_id, title, publisher_name, publication_year, language)
VALUES
(1, 'To Kill a Mockingbird', 'HarperCollins', 1960, 'EN'),
(2, '1984', 'Penguin Books', 1949, 'EN'),
(3, 'One Hundred Years of Solitude', 'Harper & Row', 1967, 'ES'),
(4, 'Il Nome della Rosa', 'Bompiani', 1980, 'IT'),
(5, 'Good Omens', 'Workman', 1990, 'EN')
''')

# Insert authors
cursor.execute('''
INSERT INTO author (author_id, first_name, last_name, nationality)
VALUES
(1, 'Harper', 'Lee', 'American'),
(2, 'George', 'Orwell', 'British'),
(3, 'Gabriel', 'García Márquez', 'Colombian'),
(4, 'Umberto', 'Eco', 'Italian'),
(5, 'Terry', 'Pratchett', 'British'),
(6, 'Neil', 'Gaiman', 'British')
''')

# Insert book-author relationships
cursor.execute('''
INSERT INTO book_author (book_id, author_id)
VALUES
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(5, 5),
(5, 6),
(2, 6)
''')

conn.commit()
print("Ready!")

Ready!


### AUTO INCREMENT for KEYS

In standard SQL we can use the `CREATE SEQUENCE` statement, which creates a new sequence number generator and use that one to generate `DEFAULT` values.

In SQLite this is not supported.

So a command like
`CREATE SEQUENCE author_id_seq START 101;` will generate an error

In SQLite instead there is the `AUTOINCREMENT` for the attribute.
See below an example.



In [6]:
cursor.execute('''
CREATE TABLE book_review (
    book_id INTEGER,
    review_id INTEGER PRIMARY KEY AUTOINCREMENT,
    comment text,
    FOREIGN KEY (book_id) REFERENCES book(book_id)
)
''')


# Insert book-review relationships
cursor.execute('''
INSERT INTO book_review (book_id, comment)
VALUES
(5, "good"),
(2, "nice"),
(2, "decent"),
(4, "great!"),
(4, "great!"),
(4, "not so great")
''')

conn.commit()
print("Ready!")

Ready!


In [7]:
# Select all books
cursor.execute("SELECT * FROM book_review")

# Fetch all results
result = cursor.fetchall()

# Print the book information
print("SQL results:")
for r in result:
    print(r)


SQL results:
(5, 1, 'good')
(2, 2, 'nice')
(2, 3, 'decent')
(4, 4, 'great!')
(4, 5, 'great!')
(4, 6, 'not so great')


# Testing Storage

In [8]:
from pathlib import Path

# Get the current directory
current_dir = Path.cwd()

# List all files in the current directory
print("Files in the current directory:")
for file_path in current_dir.iterdir():
    if file_path.is_file():
        print(file_path.name)

Files in the current directory:
lecture01.db


In [9]:
# Select all books
cursor.execute("SELECT * FROM book")

# Fetch all results
books = cursor.fetchall()

# Print the book information
print("List of books:")
for book in books:
    print(book)


List of books:
(1, 'To Kill a Mockingbird', 'HarperCollins', 1960, 'EN')
(2, '1984', 'Penguin Books', 1949, 'EN')
(3, 'One Hundred Years of Solitude', 'Harper & Row', 1967, 'ES')
(4, 'Il Nome della Rosa', 'Bompiani', 1980, 'IT')
(5, 'Good Omens', 'Workman', 1990, 'EN')


In [10]:
# Execute a query to select all books
cursor.execute("SELECT * FROM book")

# Fetch all results
books = cursor.fetchall()

# Print the books
print("List of all books:")
# We use the python string format  syntax here to make the output more readable
# see: https://docs.python.org/3/tutorial/inputoutput.html#tut-f-strings
for book in books:
    print(f"ID: {book[0]}, Title: {book[1]}, Publisher: {book[2]}, Year: {book[3]}, Language: {book[4]}")

List of all books:
ID: 1, Title: To Kill a Mockingbird, Publisher: HarperCollins, Year: 1960, Language: EN
ID: 2, Title: 1984, Publisher: Penguin Books, Year: 1949, Language: EN
ID: 3, Title: One Hundred Years of Solitude, Publisher: Harper & Row, Year: 1967, Language: ES
ID: 4, Title: Il Nome della Rosa, Publisher: Bompiani, Year: 1980, Language: IT
ID: 5, Title: Good Omens, Publisher: Workman, Year: 1990, Language: EN


In [11]:
cursor.close()

##  Remember to Save your Database file
Colab Command to Download the SQLite file you are using,  in this way you have a `backup`

In [12]:
from google.colab import files
files.download('lecture01.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>